In [1]:
import torch
from clustering import KNN
import pandas as pd

#load data from file
train_df=pd.read_csv("data/train.csv")
test_df=pd.read_csv("data/test.csv")

## load the embeddings from saved file
gpt_train_emb = torch.load('out/gpt_30379.pt', map_location="cpu")
gpt_test_emb = torch.load('out/gpt_17723.pt', map_location="cpu")

bert_train_emb = torch.load('out/bert_30379.pt', map_location="cpu")
bert_test_emb = torch.load('out/bert_17723.pt', map_location="cpu")

codebert_train_emb = torch.load('out/codebert_30379.pt', map_location="cpu")
codebert_test_emb = torch.load('out/codebert_17723.pt', map_location="cpu")

roberta_train_emb = torch.load('out/roberta_30379.pt', map_location="cpu")
roberta_test_emb = torch.load('out/roberta_17723.pt', map_location="cpu")

In [2]:
train_df["combine"]=train_df["task"]+"@"+train_df["language"]
test_df["combine"]=test_df["task"]+"@"+test_df["language"]

In [3]:
train_df.head(5)

,Unnamed: 0,task,language,code,combine
0,1,Call-an-object-method,Ada,package body My_Class is\n procedure P...,Call-an-object-method@Ada
1,2,Call-an-object-method,Ada,package Other_Class is\n type Object i...,Call-an-object-method@Ada
2,3,Call-an-object-method,Ada,package My_Class is\n type Object is t...,Call-an-object-method@Ada
3,5,Call-an-object-method,Pike,"module.func();\nmodule[""func""]();\n",Call-an-object-method@Pike
4,6,Call-an-object-method,Pike,function func = obj->method;\nfunc();\n,Call-an-object-method@Pike


In [4]:
from sklearn.preprocessing import LabelEncoder

train_label_lang = list(train_df['combine'])
test_label_lang =list(test_df['combine']) 
all_label=train_label_lang + test_label_lang

print(len(train_label_lang),len(test_label_lang),len(all_label))


le = LabelEncoder()
le=le.fit(all_label)

train_label_encoding=le.transform(train_label_lang)
test_label_encoding=le.transform(test_label_lang)

train_y = torch.tensor(train_label_encoding)
test_y = torch.tensor(test_label_encoding)

30379 17723 48102


In [5]:
knn_gpt_combined = KNN(gpt_train_emb, train_y)
knn_bert_combined = KNN(bert_train_emb, train_y)
knn_codebert_combined = KNN(codebert_train_emb, train_y)
knn_roberta_combined = KNN(roberta_train_emb, train_y)

In [6]:
import prediction
# import importlib 
# importlib.reload(prediction)

In [7]:
pred_y_gpt = prediction.predict_one(gpt_test_emb, knn_gpt_combined, test_y,le)

Time elapsed: 0.30 seconds, Data predicted:0
Time elapsed: 108.05 seconds, Data predicted:5000
Time elapsed: 1904.29 seconds, Data predicted:10000
Time elapsed: 751.57 seconds, Data predicted:15000
Accuracy of combined labels prediction: 4.72%
Total Number: 17723
All right: 836
Language correct: 5545
Task correct: 271


In [8]:
pred_y_bert = prediction.predict_one(bert_test_emb, knn_bert_combined, test_y,le)

Time elapsed: 0.24 seconds, Data predicted:0
Time elapsed: 1741.93 seconds, Data predicted:5000
Time elapsed: 108.67 seconds, Data predicted:10000
Time elapsed: 108.87 seconds, Data predicted:15000
Accuracy of combined labels prediction: 10.91%
Total Number: 17723
All right: 1933
Language correct: 6672
Task correct: 779


In [9]:
pred_y_codebert = prediction.predict_one(codebert_test_emb, knn_codebert_combined, test_y,le)

Time elapsed: 0.26 seconds, Data predicted:0
Time elapsed: 1739.61 seconds, Data predicted:5000
Time elapsed: 1824.25 seconds, Data predicted:10000
Time elapsed: 6740.07 seconds, Data predicted:15000
Accuracy of combined labels prediction: 7.66%
Total Number: 17723
All right: 1358
Language correct: 6706
Task correct: 351


In [10]:
pred_y_roberta = prediction.predict_one(roberta_test_emb, knn_roberta_combined, test_y,le)

Time elapsed: 0.24 seconds, Data predicted:0
Time elapsed: 18509.95 seconds, Data predicted:5000
Time elapsed: 7395.50 seconds, Data predicted:10000
Time elapsed: 169.84 seconds, Data predicted:15000
Accuracy of combined labels prediction: 0.01%
Total Number: 17723
All right: 1
Language correct: 236
Task correct: 19
